# Development Notebook

In this Notebook we will develop the functions used in our Streamlit EV Adoption Tracker APP

In [16]:
import pandas as pd

In [17]:
ev_url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"

In [18]:
ev_df = pd.read_csv(ev_url)
ev_df.head()


,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [19]:
# loop through the columns and display all the unique classes for categorical columns and basic stats for numerical columns
for col in ev_df.columns:
    print(f'Column: {col}')
    if ev_df[col].dtype == "object":
        print(f'{col}:{ev_df[col].unique()}')
        print("\n")
    else:
        print(f'{col}:{ev_df[col].describe()}')
        print("\n")


Column: region
region:['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']


Column: category
category:['Historical']


Column: parameter
parameter:['EV sales' 'EV stock share' 'EV sales share' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']


Column: mode
mode:['Cars']


Column: powertrain
powertrain:['BEV' 'EV' 'PHEV' 'FCEV']


Column: year
year:count    3798.000000
mean     2017.369932
std         3.809226
min      2010.000000
25%      20

In [20]:
# drop the columns that are not needed category, mode unit
ev_df = ev_df.drop(columns=["category", "mode", "unit"])

In [21]:
# budling a filter to datafram where parameter is EV sales
ev_sales_df = ev_df[ev_df["parameter"] == "EV sales"]
ev_sales_share_df = ev_df[ev_df["parameter"] == "EV sales share"]

#drop the parameter column on both dataframes
ev_df = ev_df.drop(columns=["parameter"])
ev_sales_share_df = ev_sales_share_df.drop(columns=["parameter"])

In [22]:
ev_sales_df.head()

,region,parameter,powertrain,year,value
0,Australia,EV sales,BEV,2011,49.0
6,Australia,EV sales,PHEV,2012,80.0
9,Australia,EV sales,BEV,2012,170.0
10,Australia,EV sales,BEV,2013,190.0
13,Australia,EV sales,PHEV,2013,100.0


In [23]:
# drop the powertrain column
ev_sales_share_df = ev_sales_share_df.drop(columns=["powertrain"])


In [24]:
ev_sales_share_df.head()

,region,year,value
2,Australia,2011,0.0065
7,Australia,2012,0.0300
12,Australia,2013,0.0340
19,Australia,2014,0.1600
24,Australia,2015,0.2000


In [25]:
# create a function that load the data from the a specific url, bulding a filter of sales share and sales per region an creates two dataframes
def load_data():
    url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"
    ev_df = pd.read_csv(url)
    ev_sales_df = ev_df[ev_df["parameter"] == "EV sales"]
    ev_sales_share_df = ev_df[ev_df["parameter"] == "EV sales share"]
    #drop the columns that are not needed from both dataframes category, mode, unit and build a filter where parameter is EV sales and EV sales share
    ev_sales_df = ev_sales_df.drop(columns=["category", "mode", "unit", "parameter"])
    ev_sales_share_df = ev_sales_share_df.drop(columns=["category", "mode", "unit", "parameter", "powertrain"])
    return ev_sales_df, ev_sales_share_df

In [26]:
(a,b) = load_data()

In [27]:
display(a.head())
display(b.head())

,region,powertrain,year,value
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0


,region,year,value
2,Australia,2011,0.0065
7,Australia,2012,0.0300
12,Australia,2013,0.0340
19,Australia,2014,0.1600
24,Australia,2015,0.2000


### top sales by country
this function will take in the sales df and return a sorted df with the top 10 countries with most sales

In [28]:
# filter the df to only include the year 2023
ev_sales_df_2023 = ev_sales_df[ev_sales_df["year"] == year]

# drop the year column
ev_sales_df_2023 = ev_sales_df_2023.drop(columns=["year"])

# filter out regions that are not countries: World, OECD, G20, G7, EU, EU27, EU28, Rest of World, Europe, Asia, North America, South America, Africa, Middle East, Oceania
ev_sales_df_2023 = ev_sales_df_2023[~ev_sales_df_2023["region"].isin(["World", "OECD", "G20", "G7", "EU", "EU27", "EU28", "Rest of World", "Europe", "Asia", "North America", "South America", "Africa", "Middle East", "Oceania"])]

# aggregate the sales by region
ev_sales_df_2023_agg = ev_sales_df_2023.groupby("region")["value"].sum().reset_index()

In [29]:
#rename the region column to country
ev_sales_df_2023_agg = ev_sales_df_2023_agg.rename(columns={"region": "country"})

# rename the column value to sales
ev_sales_df_2023_agg = ev_sales_df_2023_agg.rename(columns={"value": "sales"})

#convert the sales column into millions
ev_sales_df_2023_agg["sales_(m)"] = ev_sales_df_2023_agg["sales"] / 1000000

#round the sales column to 2 decimal places
ev_sales_df_2023_agg["sales_(m)"] = ev_sales_df_2023_agg["sales_(m)"].round(2)

#drop the sales column
ev_sales_df_2023_agg = ev_sales_df_2023_agg.drop(columns=["sales"])

#sort the df by the sales column in descending order
ev_sales_df_2023_agg = ev_sales_df_2023_agg.sort_values(by="sales_(m)", ascending=False)

#reset the index and make it start from 1 
ev_sales_df_2023_agg = ev_sales_df_2023_agg.reset_index(drop=True)
ev_sales_df_2023_agg.index = ev_sales_df_2023_agg.index + 1

# return the top 10 countries
return ev_sales_df_2023_agg.head(10)

,country,sales_(m)
1,China,8.10
2,USA,1.39
3,Germany,0.70
4,France,0.47
5,United Kingdom,0.45
6,Belgium,0.19
7,Sweden,0.17
8,Canada,0.17
9,Netherlands,0.16
10,Korea,0.14


In [43]:
def top10_sales_by_country(ev_sales_df, year=2023):

    # filter the df to only include the year 2023
    ev_sales_df_2023 = ev_sales_df[ev_sales_df["year"] == year]

    # drop the year column
    ev_sales_df_2023 = ev_sales_df_2023.drop(columns=["year"])

    # filter out regions that are not countries: World, OECD, G20, G7, EU, EU27, EU28, Rest of World, Europe, Asia, North America, South America, Africa, Middle East, Oceania
    ev_sales_df_2023 = ev_sales_df_2023[~ev_sales_df_2023["region"].isin(["World", "OECD", "G20", "G7", "EU", "EU27", "EU28", "Rest of World", "Europe", "Asia", "North America", "South America", "Africa", "Middle East", "Oceania"])]

    # aggregate the sales by region
    ev_sales_df_2023_agg = ev_sales_df_2023.groupby("region")["value"].sum().reset_index()
    #rename the region column to country
    ev_sales_df_2023_agg = ev_sales_df_2023_agg.rename(columns={"region": "country"})
    # rename the column value to sales
    ev_sales_df_2023_agg = ev_sales_df_2023_agg.rename(columns={"value": "sales"})

    #convert the sales column into millions
    ev_sales_df_2023_agg["sales_(m)"] = ev_sales_df_2023_agg["sales"] / 1000000

    #round the sales column to 2 decimal places
    ev_sales_df_2023_agg["sales_(m)"] = ev_sales_df_2023_agg["sales_(m)"].round(2)

    #drop the sales column
    ev_sales_df_2023_agg = ev_sales_df_2023_agg.drop(columns=["sales"])

    #sort the df by the sales column in descending order
    ev_sales_df_2023_agg = ev_sales_df_2023_agg.sort_values(by="sales_(m)", ascending=False)

    #reset the index and make it start from 1 
    ev_sales_df_2023_agg = ev_sales_df_2023_agg.reset_index(drop=True)
    ev_sales_df_2023_agg.index = ev_sales_df_2023_agg.index + 1

    # return the top 10 countries
    return ev_sales_df_2023_agg.head(10)